Bayesian Optimization & NN Opt
---

In [1]:
from bayes_opt import BayesianOptimization
from nnopt.nnopt import Optimizer
import time
import numpy as np
import gym


In [2]:
ll = gym.make("MountainCarContinuous-v0")
print(ll.action_space)
print(ll.observation_space)

def black_box_function(a, b):
    """Function with unknown internals we wish to maximize.

    This is just serving as an example, for all intents and
    purposes think of the internals of this function, i.e.: the process
    which generates its output values, as unknown.
    """
    transform = np.array([[a, b]])
    
    sc = 0
    for _ in range(4):
        s = ll.reset()
        done = False
        while not done:
            #ll.render()
            s, r, done, _ = ll.step(transform @ s)
            sc += r

    return sc / 4

#print(black_box_function(-221.59328, 28.267517))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
Box(1,)
Box(2,)


/usr/local/lib/python3.6/site-packages/gym/envs/registration.py:14: DeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [3]:
Nopt = Optimizer(black_box_function, N=2, surrogate_hidden_layer=1000, model_hidden_layer=400, memory_hidden_layer=400,
                 Rn=[(-10, 10), (-10, 10)]) #(-10, 10), (-10, 10), (-10, 10), (-10, 10)])

Bopt = BayesianOptimization(
    f=black_box_function,
    pbounds={'a': (-10, 10), 'b': (-10, 10)#, 'c': (-10, 10), 
             #'d': (-10, 10), 'e': (-10, 10), 'f': (-10, 10)},
            },
    random_state=np.random.randint(10),
)

Bayesian
---

In [4]:
timestamp = time.time()
Bopt.maximize(
    init_points=40,
    n_iter=5,
)

print("Bayesian took: {}".format(time.time() - timestamp))

|   iter    |  target   |     a     |     b     |
-------------------------------------------------
|  1        | -65.47    |  1.016    |  4.163    |
|  2        | -193.9    | -4.182    |  0.2166   |
|  3        | -4.569e+0 |  7.859    |  7.926    |
|  4        | -567.0    | -7.488    | -5.855    |
|  5        | -901.5    | -8.971    | -1.184    |
|  6        | -1.016e+0 | -9.402    | -0.8633   |
|  7        | -631.1    |  2.983    | -4.43     |
|  8        | -758.0    |  3.525    |  1.817    |
|  9        | -979.2    | -9.52     |  1.177    |
|  10       | -252.8    | -4.815    | -1.698    |
|  11       | -333.7    | -4.329    |  3.863    |
|  12       | -25.2     | -1.191    | -6.863    |
|  13       | -49.77    |  0.893    |  5.606    |
|  14       | -157.3    | -3.873    | -5.561    |
|  15       |  55.44    | -2.241    |  8.728    |
|  16       | -5.484e+0 |  9.52     |  3.448    |
|  17       | -4.546e+0 |  8.057    |  6.915    |
|  18       | -71.93    | -2.44     | -8.156    |


In [5]:
timestamp = time.time()
best, target = Nopt.run(random=50, optimization=10, fitting=10000, verbose=True)

print("Best", best, "target", target)
print(black_box_function(*best))
print("NNopt took:", time.time() - timestamp)

sample [9.420309 9.439905] target -5756.810204662098
sample [ 8.190327  -1.7384768] target -4372.012498186571
sample [-2.6557112  7.5721188] target 23.539262239088373
sample [ 0.7985449 -6.5833545] target -24.95415590581313
sample [-1.3427467 -2.7076244] target -30.618542961439726
sample [1.4698715 2.2531052] target -149.11301479145774
sample [-3.414216   5.6880207] target -84.46931848403598
sample [ 3.4243727 -9.761043 ] target -783.4537995962372
sample [6.514387 2.356079] target -3234.9076755793003
sample [ 2.1313715 -6.079526 ] target -286.5034334495592
sample [-1.3669634 -4.0932703] target -31.323286429441062
sample [-9.864459  7.062504] target -1277.104218929992
sample [ 3.6365585 -2.3139596] target -1006.6134985491401
sample [-7.604556  -0.8005123] target -643.3178639704153
sample [-6.6045427  7.584587 ] target -662.2299344935817
sample [3.3784084 1.067522 ] target -733.9708833150924
sample [1.4477873 7.0994015] target -147.91906546171876
sample [9.257956 9.477764] target -5845.2